## Part 4 - Deploy Model

**Connect to Azure ML workspace and get deployed model**

In [22]:
from azureml.core import Workspace

workspace = Workspace.from_config()
model = workspace.models['cifar-classifier']

**Define score file and environment (inference configuration)**

In [23]:
%pycat deploy/score.py

In [24]:
%pycat deploy/env.yml

In [7]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='deploy',
                                   runtime='python',
                                   entry_script='score.py',
                                   conda_file='env.yml')

**Deploy as local web service**

In [25]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import LocalWebservice
from azureml.core import Model

local_service_name = 'local-cifar-classifier'
local_config = LocalWebservice.deploy_configuration(port=3000)

try:
    local_service = Webservice(workspace, name=local_service_name)
    if local_service:
        local_service.delete()
except WebserviceException as e:
    print()

local_service = Model.deploy(workspace, local_service_name, [model], inference_config, local_config)
local_service.wait_for_deployment()


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry johneast3c830537.azurecr.io
Logging into Docker registry johneast3c830537.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM johneast3c830537.azurecr.io/azureml/azureml_4704ad523163591b97b7ba59c858ae17
 ---> 4e7642992776
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> e53bf006057d
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjE1YWU5Y2I2LTk1YzEtNDgzZC1hMGUzLWIxYTFhM2IwNjMyNCIsInJlc291cmNlR3JvdXBOYW1lIjoiam9obiIsImFjY291bnROYW1lIjoiam9obi1lYXN0Iiwid29ya3NwYWNlSWQiOiI0ZWI1MmJkOS01OTQzLTQ3ODEtODIyYi1kNzBiMzQ2ODc3ODkifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 26343fb8bb87
 ---> be59a75db248
Step 4/5 : RUN mv '/var/azureml-app/tmpfauyk3hz.py' /var/azureml-app/main.py
 ---> Running in 1b76095ada08
 ---> 4051b1fb3409
Step 5/5 : CMD ["runsvdir","/var/runit"]
 

**Test local service**

In [26]:
from io import BytesIO
import urllib.request
import base64
import requests 
import json
import io
from PIL import Image

def imgToBase64(img):
    '''Convert pillow image to base64-encoded image'''
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

def test_service(image_url, scoring_url):
    # Download image and convert to base 64
    with urllib.request.urlopen(image_url) as url:
        test_img = io.BytesIO(url.read())

    base64Img = imgToBase64(Image.open(test_img))
    
    # Get prediciton through endpoint
    input_data = '{\"data\": \"'+ base64Img +'\"}'
    headers = {'Content-Type':'application/json'}
    response = requests.post(scoring_url, input_data, headers=headers)
    return json.loads(response.text)

In [29]:
scoring_url = local_service.scoring_uri
image_url = 'https://www.gannett-cdn.com/-mm-/d5e2b1d154f830ff7d8b16f13178ec1d3854aa4f/c=0-265-2589-1728/local/-/media/2017/08/30/USATODAY/USATODAY/636396561829785209-XXX-KERMIT--10-OF-10-47661325.JPG'

prediction = test_service(image_url, scoring_url)
print(prediction)

{"label": "frog", "probability": "0.57938045"}


**Deploy as ACI service**

In [28]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice
from azureml.core.compute import AksCompute
from azureml.core import Model

aci_service_name = 'aci-cifar-classifier'
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4, auth_enabled=False)

try:
    aci_service = Webservice(workspace, name=aci_service_name)
    if aci_service:
        aci_service.delete()
except WebserviceException as e:
    print()

aci_service = Model.deploy(workspace, aci_service_name, [model], inference_config, aci_config)
aci_service.wait_for_deployment()

KeyboardInterrupt: 

**Test ACI service**

In [34]:
scoring_url = aci_service.scoring_uri
image_url = 'https://content.presspage.com/uploads/2431/1920_cairoa380new-135207.jpg?10000'

prediction = test_service(image_url, scoring_url)
print(prediction)

MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

**Deploy as AKS service**

In [31]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AksWebservice
from azureml.core.compute import AksCompute
from azureml.core import Model

aks_compute_target = AksCompute(workspace, 'aks-cluster-1')

aks_service_name = 'aks-cifar-classifier'
aks_config = AksWebservice.deploy_configuration(cpu_cores=2, memory_gb=4, auth_enabled=False)

try:
    aks_service = Webservice(workspace, name=aks_service_name)
    if aks_service:
        aks_service.delete()
except WebserviceException as e:
    print()

aks_service = Model.deploy(workspace, aks_service_name, [model], inference_config, aks_config, aks_compute_target)
aks_service.wait_for_deployment()


AKS service creation operation finished, operation "Succeeded"


**Test AKS service**

In [32]:
scoring_url = aks_service.scoring_uri
image_url = 'https://content.presspage.com/uploads/2431/1920_cairoa380new-135207.jpg?10000'

prediction = test_service(image_url, scoring_url)
print(prediction)

{"label": "airplane", "probability": "0.9817654"}
